In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from typing import Tuple

In [ ]:
k = np.array([0, 1, 2, 3, 4, 12])
n_k = np.array([38, 26, 8, 2, 1, 1])
print("k = ", k)
print("n_k = ", n_k)

In [ ]:
p_k = n_k / np.sum(n_k)
print("p_k = ", p_k)

In [ ]:
def poisson_plot(k: np.ndarray, p_k: np.ndarray) -> None:
    k_factorial = np.array([np.math.factorial(num) for num in k])
    plt.plot(k, np.log(p_k * k_factorial), 'o')
    plt.xlabel('k')
    plt.ylabel(r'Log($p_k$ k!)');
    plt.title('(a) Poisson plot')

In [ ]:
poisson_plot(k, p_k)

In [ ]:
def poisson_likelihood(k: np.ndarray, n_k: np.ndarray, theta: np.ndarray) -> np.ndarray:
    log_like = -theta * np.sum(n_k) + np.sum(k * n_k) * np.log(theta)
    like = np.exp(log_like)
    like /= np.max(like)
    return like

In [ ]:
theta = np.linspace(0.01, 1.4, 50)
likelihood = poisson_likelihood(k, n_k, theta)
likelihood_sans_last = poisson_likelihood(k[:-1], n_k[:-1], theta)

In [ ]:
def plot_likelihood_functions(
    theta: np.ndarray,
    likelihood: np.array,
    likelihood_sans_last: np.array) -> np.ndarray:
    plt.plot(theta, likelihood)
    plt.plot(theta, likelihood_sans_last, '--')
    plt.axhline(y=0.15)
    plt.xlabel(r'$\theta$')
    plt.ylabel('Likelihood');
    plt.legend(['All data', 'Data without k=12'])
    plt.title('(b) Likelihood functions');

In [ ]:
plot_likelihood_functions(theta, likelihood, likelihood_sans_last)

In [ ]:
def likelihood_interval(theta: np.ndarray,
                        likelihood: np.ndarray,
                        cutoff: float) -> Tuple[float, float]:
    # intersection points occur below and above the maximum likelihood estimate
    mle_index = np.argmax(likelihood)
    interp_below_max = interp1d(likelihood[:mle_index], theta[:mle_index])
    interp_above_max = interp1d(likelihood[mle_index:], theta[mle_index :])
    lower_int = np.round(interp_below_max(cutoff).flatten()[0], 2)
    upper_int = np.round(interp_above_max(cutoff).flatten()[0], 2)
    return (lower_int, upper_int)

In [ ]:
c = 0.15
print(f'Approximate 95% CI all data = {likelihood_interval(theta, likelihood, c)}')
print(f'Approximate 95% CI data without k = 12 = {likelihood_interval(theta, likelihood_sans_last, c)}')

In [ ]:
def poisson_mle(k: np.ndarray, n_k: np.ndarray) -> float:
    mle = np.sum(k * n_k) / np.sum(n_k)
    return mle

In [ ]:
theta_hat = poisson_mle(k, n_k)
theta_hat_sans_last = poisson_mle(k[:-1], n_k[:-1])
print(f'MLE theta_hat all data = ', np.round(theta_hat, 2))
print(f'MLE theta_hat data without k = 12 = ', np.round(theta_hat_sans_last, 2))